In [1]:
using Revise

In [4]:
import MatrixCompletion

┌ Info: Precompiling MatrixCompletion [e3a21e2c-a350-11e9-1f94-57957a131fd6]
└ @ Base loading.jl:1186


In [5]:
a = MatrixCompletion.ADMM()

MatrixCompletion.ADMM(0.5, 0.0005, 0.3, 1.618)

In [ ]:
a(1,1,1,1)

In [ ]:
b = MatrixCompletion.testPoisson()

In [ ]:
b(1,1,1,1)